In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

df1 = pd.read_csv('data.csv')
df2 = pd.read_csv('data-1.csv')

# Combine datasets
df_combined = pd.concat([df1, df2], ignore_index=True)


# Split into train and test (70/30 split)
train_df, test_df = train_test_split(
    df_combined, 
    test_size=0.3, 
    random_state=42, 
    stratify=df_combined['species']
)

train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)

In [5]:
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    classification_report, confusion_matrix, roc_auc_score
)
from sklearn.preprocessing import label_binarize
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

def poison_data(df, poison_percentage, method='random', seed=42):
    """
    Poison training data by corrupting features
    
    Args:
        df: Training dataframe
        poison_percentage: Percentage of data to poison (0-100)
        method: 'random' (random noise) or 'targeted' (label flipping)
        seed: Random seed for reproducibility
    
    Returns:
        Poisoned dataframe and metadata
    """
    np.random.seed(seed)
    df_poisoned = df.copy()
    n_samples = len(df_poisoned)
    n_poison = int(n_samples * poison_percentage / 100)
    
    if n_poison == 0:
        return df_poisoned, {'n_poisoned': 0, 'poison_percentage': 0, 'poison_indices': []}
    
    # Select random indices to poison
    poison_indices = np.random.choice(n_samples, n_poison, replace=False)
    
    feature_cols = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
    
    if method == 'random':
        # Add random noise to features
        for idx in poison_indices:
            for col in feature_cols:
                col_mean = df[col].mean()
                col_std = df[col].std()
                noise = np.random.normal(col_mean, col_std * 2)
                df_poisoned.loc[idx, col] = max(0, noise)
    
    elif method == 'targeted':
        for idx in poison_indices:
            current_species = df_poisoned.iloc[idx]['species']
            all_species = ['setosa', 'versicolor', 'virginica']
            all_species.remove(current_species)
            df_poisoned.loc[idx, 'species'] = np.random.choice(all_species)
    
    poisoning_info = {
        'n_poisoned': n_poison,
        'poison_indices': poison_indices.tolist(),
        'poison_percentage': poison_percentage,
        'method': method
    }
    
    return df_poisoned, poisoning_info

def detect_outliers_zscore(df, threshold=3):
    """
    Detect potential poisoned samples using Z-score method
    """
    feature_cols = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
    z_scores = np.abs(stats.zscore(df[feature_cols]))
    outlier_mask = (z_scores > threshold).any(axis=1)
    outlier_indices = df[outlier_mask].index.tolist()
    
    return outlier_indices, len(outlier_indices)

def train_model_with_mlflow(
    train_df, 
    test_df, 
    poison_percentage, 
    poison_method='random',
    experiment_name='iris_poisoning_experiment'
):
    """
    Train model on poisoned data and log everything to MLflow
    """
    mlflow.set_tracking_uri("http://34.67.15.13:8100")
    mlflow.set_experiment(experiment_name)
    
    # Encode species to numeric
    species_map = {'setosa': 0, 'versicolor': 1, 'virginica': 2}
    train_df['species_num'] = train_df['species'].map(species_map)
    test_df['species_num'] = test_df['species'].map(species_map)
    
    # Separate features and target
    feature_cols = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
    X_train = train_df[feature_cols]
    y_train = train_df['species_num']
    X_test = test_df[feature_cols]
    y_test = test_df['species_num']
    
    # Detect outliers
    outlier_indices, n_outliers = detect_outliers_zscore(train_df)
    outlier_pct = (n_outliers / len(train_df)) * 100
    
    with mlflow.start_run(run_name=f"{poison_method}_{poison_percentage}pct"):
        
        # Log parameters
        mlflow.log_param("poison_percentage", poison_percentage)
        mlflow.log_param("poison_method", poison_method)
        mlflow.log_param("train_samples", len(X_train))
        mlflow.log_param("test_samples", len(X_test))
        mlflow.log_param("n_features", len(feature_cols))
        mlflow.log_param("model_type", "RandomForest")
        mlflow.log_param("n_estimators", 1)
        mlflow.log_param("max_depth", 2)
        mlflow.log_param("outliers_detected", n_outliers)
        mlflow.log_param("outlier_percentage", f"{outlier_pct:.2f}%")
        
        # Train model
        clf = RandomForestClassifier(
            n_estimators=1,
            max_depth=2,
            random_state=42,
            min_samples_split=5,
            min_samples_leaf=2
        )
        clf.fit(X_train, y_train)
        
        # Training predictions
        y_train_pred = clf.predict(X_train)
        y_train_proba = clf.predict_proba(X_train)
        
        # Test predictions
        y_test_pred = clf.predict(X_test)
        y_test_proba = clf.predict_proba(X_test)
        
        # Calculate metrics
        train_accuracy = accuracy_score(y_train, y_train_pred)
        test_accuracy = accuracy_score(y_test, y_test_pred)
        
        train_f1 = f1_score(y_train, y_train_pred, average='macro')
        test_f1 = f1_score(y_test, y_test_pred, average='macro')
        
        train_precision = precision_score(y_train, y_train_pred, average='macro')
        test_precision = precision_score(y_test, y_test_pred, average='macro')
        
        train_recall = recall_score(y_train, y_train_pred, average='macro')
        test_recall = recall_score(y_test, y_test_pred, average='macro')
        
        # Overfitting gap
        overfitting_gap = train_accuracy - test_accuracy
        
        # Multi-class AUC
        y_test_bin = label_binarize(y_test, classes=[0, 1, 2])
        test_auc = roc_auc_score(y_test_bin, y_test_proba, average='macro', multi_class='ovr')
        
        # Log metrics
        mlflow.log_metric("train_accuracy", train_accuracy)
        mlflow.log_metric("test_accuracy", test_accuracy)
        mlflow.log_metric("train_f1_score", train_f1)
        mlflow.log_metric("test_f1_score", test_f1)
        mlflow.log_metric("train_precision", train_precision)
        mlflow.log_metric("test_precision", test_precision)
        mlflow.log_metric("train_recall", train_recall)
        mlflow.log_metric("test_recall", test_recall)
        mlflow.log_metric("test_auc", test_auc)
        mlflow.log_metric("overfitting_gap", overfitting_gap)
        
        # Per-class metrics
        report = classification_report(y_test, y_test_pred, output_dict=True, 
                                      target_names=['setosa', 'versicolor', 'virginica'])
        
        for species in ['setosa', 'versicolor', 'virginica']:
            mlflow.log_metric(f"{species}_precision", report[species]['precision'])
            mlflow.log_metric(f"{species}_recall", report[species]['recall'])
            mlflow.log_metric(f"{species}_f1", report[species]['f1-score'])
        
        # Feature importance
        feature_importance = dict(zip(feature_cols, clf.feature_importances_))
        mlflow.log_dict(feature_importance, "feature_importance.json")
        
        # Confusion matrix
        cm = confusion_matrix(y_test, y_test_pred)
        mlflow.log_dict({"confusion_matrix": cm.tolist()}, "confusion_matrix.json")
        
        # Classification report
        mlflow.log_dict(report, "classification_report.json")
        
        # Log model
        input_example = X_train.sample(min(5, len(X_train)), random_state=42)
        mlflow.sklearn.log_model(
            clf,
            "model",
            input_example=input_example,
            registered_model_name="iris_poisoned_model"
        )
        
        # Print summary
        print(f"\n{'='*70}")
        print(f"Poison: {poison_percentage}% | Method: {poison_method}")
        print(f"{'='*70}")
        print(f"Train Accuracy:     {train_accuracy:.4f}")
        print(f"Test Accuracy:      {test_accuracy:.4f}")
        print(f"Test F1 Score:      {test_f1:.4f}")
        print(f"Test AUC:           {test_auc:.4f}")
        print(f"Overfitting Gap:    {overfitting_gap:.4f}")
        print(f"Outliers Detected:  {n_outliers} ({outlier_pct:.2f}%)")
        print(f"{'='*70}\n")
        
        return {
            'run_id': mlflow.active_run().info.run_id,
            'poison_percentage': poison_percentage,
            'poison_method': poison_method,
            'test_accuracy': test_accuracy,
            'test_f1': test_f1,
            'test_auc': test_auc,
            'overfitting_gap': overfitting_gap,
            'outliers_detected': n_outliers
        }

def run_all_experiments():
    """
    Run complete poisoning experiment suite
    """    
    # Load data
    train_df = pd.read_csv('train.csv')
    test_df = pd.read_csv('test.csv')
    
    print(f"Loaded training data: {len(train_df)} samples")
    print(f"Loaded test data: {len(test_df)} samples\n")
    
    # Experiment configurations
    poison_levels = [0, 5, 10, 20, 50]
    poison_methods = ['random', 'targeted']
    
    results = []
    
    for method in poison_methods:
        for poison_pct in poison_levels:
            print(f"Experiment: {poison_pct}% poisoning ({method})")
            
            # Poison training data
            if poison_pct == 0:
                train_poisoned = train_df.copy()
                print("Clean baseline (no poisoning)")
            else:
                train_poisoned, poison_info = poison_data(
                    train_df, 
                    poison_pct, 
                    method=method
                )
                print(f"Poisoned {poison_info['n_poisoned']} training samples")
            
            # Train and evaluate
            result = train_model_with_mlflow(
                train_poisoned, 
                test_df, 
                poison_pct, 
                method
            )
            results.append(result)
    
    # Print summary
    print("\n" + "="*70)
    print("EXPERIMENT SUMMARY")
    print("="*70)
    print(f"{'Poison %':<12}{'Method':<12}{'Test Acc':<12}{'F1':<10}{'AUC':<10}{'Overfit'}")
    print("-"*70)
    
    for r in results:
        print(f"{r['poison_percentage']:<12}{r['poison_method']:<12}"
              f"{r['test_accuracy']:<12.4f}{r['test_f1']:<10.4f}"
              f"{r['test_auc']:<10.4f}{r['overfitting_gap']:.4f}")
    
    print("="*70)
    print("\nAll experiments completed!")
    print("📊 View detailed results:")    
    return results

if __name__ == "__main__":
    # Make sure data files exist
    import os
    if not os.path.exists('train.csv') or not os.path.exists('test.csv'):
        print("❌ train.csv or test.csv not found!")
        print("   Run prepare_data.py first to create these files.")
    else:
        run_all_experiments()


Loaded training data: 105 samples
Loaded test data: 45 samples

Experiment: 0% poisoning (random)
Clean baseline (no poisoning)


2025/11/16 08:47:57 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Registered model 'iris_poisoned_model' already exists. Creating a new version of this model...
2025/11/16 08:48:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris_poisoned_model, version 11
Created version '11' of model 'iris_poisoned_model'.



Poison: 0% | Method: random
Train Accuracy:     0.9714
Test Accuracy:      0.9333
Test F1 Score:      0.9333
Test AUC:           0.9556
Overfitting Gap:    0.0381
Outliers Detected:  1 (0.95%)

🏃 View run random_0pct at: http://34.67.15.13:8100/#/experiments/1/runs/49e00816ceba44379abcbb91b6a1f46e
🧪 View experiment at: http://34.67.15.13:8100/#/experiments/1
Experiment: 5% poisoning (random)
Poisoned 5 training samples


2025/11/16 08:48:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Registered model 'iris_poisoned_model' already exists. Creating a new version of this model...
2025/11/16 08:48:05 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris_poisoned_model, version 12
Created version '12' of model 'iris_poisoned_model'.



Poison: 5% | Method: random
Train Accuracy:     0.9238
Test Accuracy:      0.9333
Test F1 Score:      0.9333
Test AUC:           0.9444
Overfitting Gap:    -0.0095
Outliers Detected:  2 (1.90%)

🏃 View run random_5pct at: http://34.67.15.13:8100/#/experiments/1/runs/54bd62a559a84bb98ba0b8289eea48b1
🧪 View experiment at: http://34.67.15.13:8100/#/experiments/1
Experiment: 10% poisoning (random)
Poisoned 10 training samples


...

2025/11/16 08:48:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Registered model 'iris_poisoned_model' already exists. Creating a new version of this model...
2025/11/16 08:48:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris_poisoned_model, version 19
Created version '19' of model 'iris_poisoned_model'.



Poison: 20% | Method: targeted
Train Accuracy:     0.7714
Test Accuracy:      0.9333
Test F1 Score:      0.9333
Test AUC:           0.9385
Overfitting Gap:    -0.1619
Outliers Detected:  1 (0.95%)

🏃 View run targeted_20pct at: http://34.67.15.13:8100/#/experiments/1/runs/b11f98297123469fbffef9cb7454ce1b
🧪 View experiment at: http://34.67.15.13:8100/#/experiments/1
Experiment: 50% poisoning (targeted)
Poisoned 52 training samples


2025/11/16 08:48:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Registered model 'iris_poisoned_model' already exists. Creating a new version of this model...
2025/11/16 08:48:39 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris_poisoned_model, version 20



Poison: 50% | Method: targeted
Train Accuracy:     0.5524
Test Accuracy:      0.5333
Test F1 Score:      0.4589
Test AUC:           0.7496
Overfitting Gap:    0.0190
Outliers Detected:  1 (0.95%)

🏃 View run targeted_50pct at: http://34.67.15.13:8100/#/experiments/1/runs/98bc6a4d724c4f15b6615d00a4ba24b9
🧪 View experiment at: http://34.67.15.13:8100/#/experiments/1

EXPERIMENT SUMMARY
Poison %    Method      Test Acc    F1        AUC       Overfit
----------------------------------------------------------------------
0           random      0.9333      0.9333    0.9556    0.0381
5           random      0.9333      0.9333    0.9444    -0.0095
10          random      0.9333      0.9333    0.9444    -0.0190
20          random      0.8444      0.8410    0.8067    0.0032
50          random      0.9333      0.9333    0.9444    -0.2190
0           targeted    0.9333      0.9333    0.9556    0.0381
5           targeted    0.9333      0.9333    0.9533    -0.0095
10          targeted    0.9333  

Created version '20' of model 'iris_poisoned_model'.
